In [1]:
from glob import glob
import sys, time, os, asyncio
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from astropy.time import Time, TimeDelta
from scipy.interpolate import UnivariateSpline

from lsst_efd_client import EfdClient, resample, merge_packed_time_series, rendezvous_dataframes
from lsst.ts.idl.enums import MTM1M3

from scipy.signal import find_peaks
from scipy.interpolate import interp1d
import astropy.units as u

In [2]:
events=pd.read_csv("/home/p/pferguso/u/projects/m1m3/hp_oscillations_during_slew/data/230622_combined_events.csv")

In [13]:
events=events.sort_values("times", ascending=False).reset_index(drop=True)

In [14]:
events

,times,heights,counts,actuators,slew_state,elevation_velocity,elevation_position,iso_time
0,1.687466e+09,373.655443,5,5.0,True,0.049640,64.018060,2023-06-22 20:39:44.145
1,1.687466e+09,349.731236,4,5.0,True,0.050123,46.091264,2023-06-22 20:33:45.661
2,1.687426e+09,860.960412,4,5.0,True,-0.049481,4.438083,2023-06-22 09:29:25.055
3,1.687426e+09,769.855694,4,0.0,True,-0.049997,16.518888,2023-06-22 09:25:23.469
4,1.687420e+09,160.929933,4,5.0,True,0.050056,60.463254,2023-06-22 07:38:33.040
...,...,...,...,...,...,...,...,...
86,1.685445e+09,555.568822,4,0.0,True,0.050235,47.580942,2023-05-30 11:08:27.615
87,1.685444e+09,510.715932,4,0.0,True,-0.050017,3.131950,2023-05-30 10:46:12.110
88,1.685443e+09,425.554757,4,0.0,True,-0.049787,19.270249,2023-05-30 10:40:49.341
89,1.685441e+09,422.163210,5,0.0,True,0.049825,69.384946,2023-05-30 10:08:01.938


In [15]:
def link_from_date(timestamp):
    
    begin_timestamp=(Time(timestamp, format="unix") - TimeDelta(3, format='sec')).iso
    end_timestamp=(Time(timestamp, format="unix") + TimeDelta(3, format='sec')).iso
    
    begin_day=begin_timestamp[:10]
    begin_hour=begin_timestamp[11:13]
    begin_minute=begin_timestamp[14:16]
    begin_second=begin_timestamp[17:]
    
    end_day=end_timestamp[:10]
    end_hour=end_timestamp[11:13]
    end_minute=end_timestamp[14:16]
    end_second=end_timestamp[17:]
    
    url="https://summit-lsp.lsst.codes/chronograf/sources/1/dashboards/"
    url+="207?refresh=Paused&tempVars%5BDownsample%5D=Default&tempVars%5BFunction%5D="
    url+=f"raw&lower={begin_day}T{begin_hour}%3A{begin_minute}%3A{begin_second}Z"
    url+=f'&upper={end_day}T{end_hour}%3A{end_minute}%3A{end_second}Z'
    #url+=f"&zoomedLower={begin_day}T{begin_hour}%3A{begin_minute}%3A{begin_second}Z&zoomedUpper=2023-05-30T10%3A08%3A02.962Z"
    return url

In [16]:
events["times"][0]-2

1687466382.1445231

In [17]:
link_from_date(events["times"][15]-2)

'https://summit-lsp.lsst.codes/chronograf/sources/1/dashboards/207?refresh=Paused&tempVars%5BDownsample%5D=Default&tempVars%5BFunction%5D=raw&lower=2023-06-22T04%3A00%3A09.767Z&upper=2023-06-22T04%3A00%3A15.767Z'

In [18]:
title_str="| event number  | Time  |  elevation |  elevation velocity | hardpoint/ims dashboard  |\n|---|---|---|---|---|\n"


In [19]:
print(title_str)

| event number  | Time  |  elevation |  elevation velocity | hardpoint/ims dashboard  |
|---|---|---|---|---|



In [20]:
events.columns

Index(['times', 'heights', 'counts', 'actuators', 'slew_state',
       'elevation_velocity', 'elevation_position', 'iso_time'],
      dtype='object')

In [21]:
row_str=""
for j,i in enumerate((range(len(events)))):
    #if events["group"][i] < 3:
    row_str+=f"| {len(events)-j}  | {events['iso_time'][i]}  | "
    row_str+=f"{events['elevation_position'][i]:0.2f}  | {events['elevation_velocity'][i]:0.2f} " 
    row_str+='| <a href="'
    row_str+=f'{link_from_date(events["times"][i]-2)})'
    row_str+='" target="_blank">link to dashboard</a>|\n'

In [22]:
print(title_str+row_str)

| event number  | Time  |  elevation |  elevation velocity | hardpoint/ims dashboard  |
|---|---|---|---|---|
| 91  | 2023-06-22 20:39:44.145  | 64.02  | 0.05 | <a href="https://summit-lsp.lsst.codes/chronograf/sources/1/dashboards/207?refresh=Paused&tempVars%5BDownsample%5D=Default&tempVars%5BFunction%5D=raw&lower=2023-06-22T20%3A39%3A39.145Z&upper=2023-06-22T20%3A39%3A45.145Z)" target="_blank">link to dashboard</a>|
| 90  | 2023-06-22 20:33:45.661  | 46.09  | 0.05 | <a href="https://summit-lsp.lsst.codes/chronograf/sources/1/dashboards/207?refresh=Paused&tempVars%5BDownsample%5D=Default&tempVars%5BFunction%5D=raw&lower=2023-06-22T20%3A33%3A40.661Z&upper=2023-06-22T20%3A33%3A46.661Z)" target="_blank">link to dashboard</a>|
| 89  | 2023-06-22 09:29:25.055  | 4.44  | -0.05 | <a href="https://summit-lsp.lsst.codes/chronograf/sources/1/dashboards/207?refresh=Paused&tempVars%5BDownsample%5D=Default&tempVars%5BFunction%5D=raw&lower=2023-06-22T09%3A29%3A20.055Z&upper=2023-06-22T09%3A29%3A26.